# **Data modeling with Cassandra (By using Sparkify data)**

----

- PROJECT OBJECT

In this project, it will use Cassandra to store the great amount of users' activity data. This project can be seperated into five parts. The **first part** is about using python to extract data from all csv files and save into an "overall csv file" for further use. The **second part** is relating to design Cassandra tables based on analytics team's needs and put data in these tables. As the design philosophy in Cassandra is different than tranditional SQL database, we must develop the table based on queries the analytics teams want. Since the analytics teams gives three queries, we will create three tables for them. The **third part** is about testing. After putting data into tables, we will run the queries to test whether we can get data from Cassandra tables or not. The **fourth part** is regarding to drop table. The **fifth part** is about closing the connection with cassandra.

-------

## Part1. Extract data from CSV files and save in an "overall CSV file"

#### 1. Import Python packages 

In [36]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### 2. Creating list of filepaths to process original event csv data files

In [37]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### 3. Process the files to create an "overall CSV file" that will be used for Apache Casssandra tables

In [38]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# create dialect for following csv write            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# creating a csv file called event_datafile_new and dump data in it
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [39]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part2. design Cassandra tables and put data in these tables

#### The image below is a screenshot of event_datafile_new.csv
<img src="images/image_event_datafile_new.jpg">

#### 1. Creating a Cluster

In [40]:
# create a connection to cassandra database
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### 2. Create Keyspace

In [41]:
# create a Keyspace "sparkifycassandra" 
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifycassandra 
                    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}"""
               )

#### 4. Set Keyspace

In [42]:
# connect to the sparkifycassandra keyspace
session.set_keyspace('sparkifycassandra')

#### 5. Create tables bases on queries

##### 5-1. Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [43]:
# create a table for query1
query = "CREATE TABLE IF NOT EXISTS song_information_on_specific_session "
query = query + "(sessionId int, \
                  itemInSession int, \
                  artist text, \
                  song text, \
                  length float,\
                  PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)


# insert data into this table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_information_on_specific_session (sessionId, \
                                                                   itemInSession, \
                                                                   artist, \
                                                                   song, \
                                                                   length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

##### 5-2. Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [44]:
# create a table for query2
query = "CREATE TABLE IF NOT EXISTS song_for_specific_session_and_user"
query = query + "(sessionId int, \
                  userId int, \
                  itemInSession int, \
                  artist text, \
                  song text, \
                  firstName text, \
                  lastName text, \
                  PRIMARY KEY ((sessionId, userid), itemInSession))"
session.execute(query)     





# insert data into this table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_for_specific_session_and_user (sessionId, \
                                                                 userId, \
                                                                 itemInSession, \
                                                                 artist, \
                                                                 song, \
                                                                 firstName, \
                                                                 lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

##### 5-3. Query 3:  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [45]:
# create a table for query3
query = "CREATE TABLE IF NOT EXISTS song_listerned_by_who "
query = query + "(song text, \
                  userId int, \
                  firstName text, \
                  lastName text, \
                  PRIMARY KEY (song, userId))"

session.execute(query)


# insert data into this table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listerned_by_who (song, \
                                                    userId, \
                                                    firstName, \
                                                    lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## Part3. test whether the data is stored successfully or nor

In [46]:
# test query 1
query = "select artist, song, length from song_information_on_specific_session WHERE sessionId = 338 AND itemInSession = 4"
rows = session.execute(query)
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [47]:
# test query 2
query = "select artist, song, firstname, lastname from song_for_specific_session_and_user WHERE sessionId = 182 AND userId = 10"
rows = session.execute(query)
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [48]:
# test query 3
query = "select firstName, lastName from song_listerned_by_who WHERE song = 'All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


## Part4. Drop the tables before closing out the sessions

In [49]:
# drop the table which is created for query 1
query = "drop table song_information_on_specific_session"
session.execute(query)

# drop the table which is created for query 2
query = "drop table song_for_specific_session_and_user"
session.execute(query)

# drop the table which is created for query 3
query = "drop table song_listerned_by_who"
session.execute(query)

## Part5. Close the session and cluster connection

In [50]:
session.shutdown()
cluster.shutdown()